In [4]:
import torch
import torch.optim as optim
from torch.nn.functional import cross_entropy
import wandb

from src.RandmanFunctions import read_randman10_dataset
from src.Models import RandmanSNN
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.Training import train_loop_snn

device = 'cpu'

In [7]:
print(torch.backends.cudnn.version())

None


In [5]:
# check if GPU is available
print(torch.cuda.is_available())

False


## Create Sweep

In [ ]:
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {        
        # Dataset:
        "nb_input": {"value": 100},
        "nb_output": {"value": 10},
        "nb_steps": {"value": 50},
        "nb_data_samples": {"value": 1000},
        # SNN:
        "nb_hidden": {"value": 10},
        "learn_beta": {"value": False},        
        # Training:
        "std": {"values": [0.05, 0.1]},
        "epochs": {"value":50},
        "batch_size": {"values": [64, 256]},
        # Optimization:
        "loss_fn": {"value": "cross-entropy"},
        "optimizer": {"value": "Adam"},
        "lr": {"value": 0.01},
        "regularization": {"value": "none"},
        # Evolution Strategy:
        "nb_model_samples": {"value": 20},
        "mirror": {"value": True},
    },
}

sweep_id = wandb.sweep(sweep_config, project="ES-Randman10")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 78qhh6zn
Sweep URL: https://wandb.ai/yixing/ES-Randman10/sweeps/78qhh6zn


## 🧹Sweep 

In [4]:
def train_snn(config=None):
    with torch.no_grad(), wandb.init(
        config=config
    ) as run:
        config = wandb.config
        # initialize Evolution Strategy instance
        es_model = ESModel(
            RandmanSNN,
            config.nb_input,
            config.nb_hidden,
            config.nb_output,
            0.95,
            sample_size=config.nb_model_samples,
            param_std=config.std,
            Optimizer=optim.Adam,
            lr=config.lr,
            device=device,
            mirror=config.mirror,
        )

        # load dataset
        train_loader, val_loader = read_randman10_dataset(
            "data/randman_10_dataset.pt", batch_size=config.batch_size
        )

        # epochs
        for epoch in range(config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(
            es_model, train_loader, val_loader, cross_entropy, device, run
            )

In [5]:
# set sweep_id if picking up a sweep:
# sweep_id = ryhbaq55
wandb.agent(sweep_id, train_snn)

wandb: Agent Starting Run: 6ri90zn0 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 2
wandb: 	learn_beta: False
wandb: 	loss_fn: cross-entropy
wandb: 	lr: 0.01
wandb: 	mirror: True
wandb: 	nb_data_samples: 1000
wandb: 	nb_hidden: 10
wandb: 	nb_input: 100
wandb: 	nb_model_samples: 20
wandb: 	nb_output: 10
wandb: 	nb_steps: 50
wandb: 	optimizer: Adam
wandb: 	regularization: none
wandb: 	std: 0.05
wandb: Currently logged in as: yixing to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 0
-------------------------------
batch 0, loss: 2.332244, accuracy: 12.5%
Test Error: 
Accuracy: 11.7%, Avg loss: 2.331179 

batch 1, loss: 2.342872, accuracy: 6.2%
Test Error: 
Accuracy: 11.1%, Avg loss: 2.328147 

batch 2, loss: 2.351147, accuracy: 7.8%
Test Error: 
Accuracy: 11.3%, Avg loss: 2.322961 

batch 3, loss: 2.321573, accuracy: 6.2%
Test Error: 
Accuracy: 11.0%, Avg loss: 2.319260 

batch 4, loss: 2.310365, accuracy: 7.8%
Test Error: 
Accuracy: 11.5%, Avg loss: 2.317127 

batch 5, loss: 2.283870, accuracy: 17.2%
Test Error: 
Accuracy: 11.3%, Avg loss: 2.314515 

batch 6, loss: 2.337767, accuracy: 9.4%
Test Error: 
Accuracy: 11.1%, Avg loss: 2.313232 

batch 7, loss: 2.326154, accuracy: 9.4%
Test Error: 
Accuracy: 10.8%, Avg loss: 2.312222 

batch 8, loss: 2.309592, accuracy: 10.9%
Test Error: 
Accuracy: 10.8%, Avg loss: 2.310760 

batch 9, loss: 2.311606, accuracy: 9.4%
Test Error: 
Accuracy: 10.3%, Avg loss: 2.310198 

batch 10, loss: 2.304059, accuracy: 17.2%
Test 

train_acc,▁▁▁▁▂▂▂▃▂▂▃▃▃▂▂▂▂▂▃▃▃▅▅▄▅▆▆▇▆▅▇▅▅▆▆▆▆█▆▇
train_average_neuron_spikes,▂▂▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▆██
train_loss,████████████▇▇▇▇▆▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▆▄▃▃▃▄▃▁
train_spike_percentage,▃▁▁▁▂▂▂▂▂▂▃▃▃▃▄▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█
val_acc,▁▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█▇▇██
val_average_neuron_spikes,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇█
val_loss,██████████▇▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁
val_spike_percentage,▃▁▁▁▁▂▂▂▂▂▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██
train_acc,0.5
train_average_neuron_spikes,1.30938
train_loss,1.53936


wandb: Agent Starting Run: b518o35p with config:
wandb: 	batch_size: 64
wandb: 	epochs: 2
wandb: 	learn_beta: False
wandb: 	loss_fn: cross-entropy
wandb: 	lr: 0.01
wandb: 	mirror: True
wandb: 	nb_data_samples: 1000
wandb: 	nb_hidden: 10
wandb: 	nb_input: 100
wandb: 	nb_model_samples: 20
wandb: 	nb_output: 10
wandb: 	nb_steps: 50
wandb: 	optimizer: Adam
wandb: 	regularization: none
wandb: 	std: 0.1


Epoch 0
-------------------------------
batch 0, loss: 2.325347, accuracy: 6.2%
Test Error: 
Accuracy: 10.0%, Avg loss: 2.325980 

batch 1, loss: 2.314633, accuracy: 3.1%
Test Error: 
Accuracy: 11.2%, Avg loss: 2.318789 

batch 2, loss: 2.300352, accuracy: 17.2%
Test Error: 
Accuracy: 11.7%, Avg loss: 2.313492 

batch 3, loss: 2.340903, accuracy: 4.7%
Test Error: 
Accuracy: 11.7%, Avg loss: 2.311439 

batch 4, loss: 2.365021, accuracy: 4.7%
Test Error: 
Accuracy: 11.2%, Avg loss: 2.310555 

batch 5, loss: 2.308555, accuracy: 12.5%
Test Error: 
Accuracy: 11.6%, Avg loss: 2.308208 

batch 6, loss: 2.296499, accuracy: 4.7%
Test Error: 
Accuracy: 11.3%, Avg loss: 2.307785 

batch 7, loss: 2.333830, accuracy: 7.8%
Test Error: 
Accuracy: 10.9%, Avg loss: 2.308995 

batch 8, loss: 2.311252, accuracy: 9.4%
Test Error: 
Accuracy: 11.1%, Avg loss: 2.307350 

batch 9, loss: 2.307514, accuracy: 9.4%
Test Error: 
Accuracy: 10.7%, Avg loss: 2.305935 

batch 10, loss: 2.307033, accuracy: 9.4%
Test Er

...

train_acc,▆▂▃▁▄▁▅▄▃▅▃▅▁▂▃▃▇▃▂▅▃▇▄▃▁▃▄▃▅▂▅▇▆▄▆▄▅▅█▆
train_average_neuron_spikes,█▇▇▆▆▅▅▅▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▆▄▇█▆▆▂▄▄▅▅▄▃▄▆▃▄▃▅▄▄▄▃▂▃▄▂▃▂▂▂▃▁▂▁▃▂▃▂▂
train_spike_percentage,█▇▇▇▆▅▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▃▃▃▂▂▂▃▁▂▂▃▃▃▃▃▄▄▅▅▄▅▆▅▅▆▆▆▆▆▆█████████▇
val_average_neuron_spikes,█▇▇▆▆▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_spike_percentage,█▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.125
train_average_neuron_spikes,0.1
train_loss,2.29325


wandb: Agent Starting Run: v2d3ock2 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 2
wandb: 	learn_beta: False
wandb: 	loss_fn: cross-entropy
wandb: 	lr: 0.01
wandb: 	mirror: True
wandb: 	nb_data_samples: 1000
wandb: 	nb_hidden: 10
wandb: 	nb_input: 100
wandb: 	nb_model_samples: 20
wandb: 	nb_output: 10
wandb: 	nb_steps: 50
wandb: 	optimizer: Adam
wandb: 	regularization: none
wandb: 	std: 0.1


Epoch 0
-------------------------------
batch 0, loss: 2.355347, accuracy: 7.0%
Test Error: 
Accuracy: 8.2%, Avg loss: 2.353222 

batch 1, loss: 2.343258, accuracy: 5.1%
Test Error: 
Accuracy: 7.5%, Avg loss: 2.349075 

batch 2, loss: 2.330523, accuracy: 5.5%
Test Error: 
Accuracy: 6.5%, Avg loss: 2.347209 

batch 3, loss: 2.347730, accuracy: 5.5%
Test Error: 
Accuracy: 6.3%, Avg loss: 2.343578 

batch 4, loss: 2.334744, accuracy: 8.6%
Test Error: 
Accuracy: 6.2%, Avg loss: 2.340262 

batch 5, loss: 2.323638, accuracy: 8.2%
Test Error: 
Accuracy: 6.0%, Avg loss: 2.336810 

batch 6, loss: 2.341033, accuracy: 5.1%
Test Error: 
Accuracy: 6.2%, Avg loss: 2.334517 

batch 7, loss: 2.341938, accuracy: 7.0%
Test Error: 
Accuracy: 6.6%, Avg loss: 2.331636 

batch 8, loss: 2.340763, accuracy: 10.2%
Test Error: 
Accuracy: 6.8%, Avg loss: 2.330243 

batch 9, loss: 2.333907, accuracy: 7.8%
Test Error: 
Accuracy: 6.5%, Avg loss: 2.329560 

batch 10, loss: 2.338249, accuracy: 9.4%
Test Error: 
Accur

Traceback (most recent call last):
  File "/tmp/ipykernel_6270/3599845855.py", line 31, in train_snn
    train_loop_snn(
  File "/home/wyx/darwin_neuron/src/Training.py", line 81, in train_loop_snn
    val_stats = val_loop_snn(es_model, val_dataloader, loss_fn, device)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wyx/darwin_neuron/src/Training.py", line 52, in val_loop_snn
    stats = run_snn_on_batch(model, x, y, loss_fn)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wyx/darwin_neuron/src/Training.py", line 25, in run_snn_on_batch
    spikes, voltages = model(x)
                       ^^^^^^^^
  File "/home/wyx/miniconda3/envs/snn/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wyx/miniconda3/envs/snn/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1747, in _call_impl
 

train_acc,▃▁▁▁▄▁▃▅▂▂▁▅▅▂▂▇▅▂▃▂▅▆▅▅▄▆█▄▇▅█▆█▄▆▇▅▂▇▅
train_average_neuron_spikes,█▅▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▅▇▅▆▆▅▅▂▅▃▄▅▅▁▃▄▃▁▃▄▃▂▂▁▃▂▂▂▁▃▃▁▂▁▁▂▂▂
train_spike_percentage,█▇▇▆▆▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▅▄▁▁▁▂▂▂▃▃▃▄▃▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████
val_average_neuron_spikes,█▇▆▆▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▇▇▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_spike_percentage,█▇▇▆▆▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.09766
train_average_neuron_spikes,0.02734
train_loss,2.30574


## Single Run


In [ ]:
import wandb

device = "cpu"

def train_snn():
    run_name = "sanity_check_1"
    config = {  # Dataset:
        "nb_input": 100,
        "nb_output": 10,
        "nb_steps": 50,
        "nb_data_samples": 1000,
        # SNN:
        "nb_hidden": 10,
        "learn_beta": False,
        # Evolution Strategy:
        "nb_model_samples": 20,
        "mirror": True,
        # Training:
        "std": 0.05,
        "epochs": 50,
        "batch_size": 256,
        # Optimization:
        "loss": "cross-entropy",
        "optimizer": "Adam",
        "lr": 0.01,
        "regularization": "none",
    }
    with torch.no_grad(), wandb.init(
        entity="DarwinNeuron", project="ES-Randman10", name=run_name, config=config
    ) as run:
        # initialize Evolution Strategy instance
        es_model = ESModel(
            RandmanSNN,
            run.config.nb_input,
            run.config.nb_hidden,
            run.config.nb_output,
            0.95,
            sample_size=run.config.nb_model_samples,
            param_std=run.config.std,
            Optimizer=optim.Adam,
            lr=run.config.lr,
            device=device,
            mirror=run.config.mirror,
        )

        # load dataset
        train_loader, val_loader = read_randman10_dataset(
            "data/randman_10_dataset.pt", batch_size=run.config.batch_size
        )

        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(
                es_model, train_loader, val_loader, cross_entropy, device, run
            )


train_snn()

## Flattened Implementation

In [ ]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters


def test_flatten():
    model = RandmanSNN(100, 10, 10, False, 0.95)
    for name, p in model.named_parameters():
        print(f"{name}: {p.shape}")
    vector = parameters_to_vector(model.parameters())
    print(vector.shape)


test_flatten()

fc1.weight: torch.Size([10, 100])
fc2.weight: torch.Size([10, 10])
torch.Size([1100])


In [ ]:
class NewESModel(EA):
    """
    The class which keeps track of all the parameters in a model, handles sampling and updates
    """
   
    def __init__(self, Model, *model_args, sample_size=20, param_std=0.05, Optimizer = optim.Adam, lr = 0.01, mirror=True, device = default_device, **kwargs):    
        """initialize ESModel with the Model class and the standard deviation of the parameters. The means of the parameters are initialized to be the initial parameters of the model.

        Args:
            Model (nn.Module): the model whose parameters are to be optimized
            param_std (float): standard deviation of the parameters, typically 0.01 to 0.05
            Optimizer (torch.optim, optional): optimizer for ES. Defaults to optim.Adam.
        """
        # self.param_dict has ('param_name', ESParam) pairs, each for one layer of the model
        self.genome = parameters_to_vector(Model(*model_args).parameters())
        self.sample_size = sample_size
        self.Model = Model
        self.model_args = model_args
        self.device = device

## Try LEAP

In [2]:
from leap_ec.simple import ea_solve

In [ ]:
def f(x):
    """A real-valued function to be optimized."""
    return sum(x)**2

ea_solve(f, bounds=[(-5.12, 5.12) for _ in range(5)], maximize=True)